In [1]:
import dicom
import os
import numpy as np
import cv2

## Generating Inputs to Function

In [2]:
BASE_DIR = '/home/radiation/RadiationTherapyDecisionSupport/data/'
PatientID = 'UCLA_PR_5'
ROI_NUM = 0

ctFilenames = [fl for fl in os.listdir(BASE_DIR + PatientID) if 'CT.' in fl]
numImages = len(ctFilenames)

sampleCTImage = dicom.read_file(BASE_DIR + PatientID + '/' + ctFilenames[0])
width = sampleCTImage.Columns
height = sampleCTImage.Rows

block_shape = (width, height, numImages) # Input to function

In [3]:
structureset = dicom.read_file(BASE_DIR + PatientID + '/structureset.dcm')
roiNumPlanes = len(structureset.ROIContourSequence[ROI_NUM].ContourSequence) # Input to function

contourData = {} # Input to function
image_orientation = {} # Input to function
image_position = {} # Input to function
pixel_spacing = {} # Input to function

for index in range(0, roiNumPlanes):
    planeContourData = np.array(structureset.ROIContourSequence[ROI_NUM].ContourSequence[index].ContourData)
    planeContourData = planeContourData.reshape(planeContourData.shape[0] // 3 , 3)
    z = planeContourData[0, 2] # All z coordinates in a plane are the same
    
    contourData[z] = planeContourData
    
    imageSOP = structureset.ROIContourSequence[ROI_NUM].ContourSequence[index].ContourImageSequence[0].ReferencedSOPInstanceUID
    imagei = dicom.read_file(BASE_DIR + PatientID + '/CT.' + imageSOP + '.dcm')
    
    image_orientation[z] = imagei.ImageOrientationPatient
    image_position[z] = imagei.ImagePositionPatient
    pixel_spacing[z] = imagei.PixelSpacing
    

## Function

In [5]:
sorted_indices = np.array(sorted(contourData))
count = 0

contour_block = np.zeros((block_shape)).astype(np.int)
roi_block = np.zeros((block_shape)).astype(np.int)

for z_coor in sorted_indices:
    row_coordinates = np.zeros((contourData[z_coor].shape[0]))
    col_coordinates = np.zeros((contourData[z_coor].shape[0]))
    
    for n in range(0, contourData[z_coor].shape[0]):
        px = contourData[z_coor][n, 0]
        py = contourData[z_coor][n, 1]
        
        xx = image_orientation[z_coor][0]
        xy = image_orientation[z_coor][1]
        yx = image_orientation[z_coor][3]
        yy = image_orientation[z_coor][4]
        
        sx = image_position[z_coor][0]
        sy = image_position[z_coor][1]
        
        delJ = pixel_spacing[z_coor][0]
        delI = pixel_spacing[z_coor][1]
        
        A = np.array([[xx * delI, yx * delJ], [xy*delI, yy*delJ]])
        b = np.array([px - sx, py - sy])
        
        v = np.linalg.solve(A, b)
        row_coordinates[count] = np.round(v[0])
        col_coordinates[count] = np.round(v[1])
        
        contour_block[row_coordinates[count], col_coordinates[count], np.argwhere(sorted_indices == z_coor)[0][0]] = 1
        
        count += 1


IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

0